In [10]:
def byte2str(n, d = 2):
    s = 'B'
    if n >= 1024:
        n/=1024
        s = 'K'
    if n >= 1024:
        n/=1024
        s = 'M'
    if n >= 1024:
        n/=1024
        s = 'G'
    if n >= 1024:
        n/=1024
        s = 'T'
    return '{:.2f}{}'.format(n,s)

In [2]:
dic = {'name': 'MIB',
'details': {'cpu':'XEON E5 6666','cpu_cores':8, 'memory':"16GB*4&2666MHz",'gpu':'RTX 2080 Ti*2'},
'timeStamp': 1932846198,
'cpu_total': 0.73,
'cpu_per_core': [0.0,0.43,0.45,0.56,0.89,0.01,0.01,0.02],
'memory_status': '562M/38.0G',
'memory_percentage': 0.07,
'top_cpu_user': 'agent_h',
'gpu_status':[
                {
                'gpu_no': 0,
                'gpu_name': 'RTX 2080 Ti',
                'gpu_memory_status': '12M/8119M',
                'gpu_util': 0.78,
                'gpu_memory': 0.23,
                'gpu_temperature': 48,
                'top_gpu_user': 'agent_n',
                },
                {
                'gpu_no': 1,
                'gpu_name': 'RTX 2080 Ti',
                'gpu_memory_status': '3434M/8119M',
                'gpu_util': 0.33,
                'gpu_memory': 0.75,
                'gpu_temperature': 78,
                'top_gpu_user': 'alien_z',
                }
            ]
}

In [1]:
import psutil

### timeStamp

### cpu_total

In [35]:
psutil.cpu_percent(interval=.5, percpu=False)

43.1

### cpu_per_core

In [3]:
psutil.cpu_percent(interval=.5, percpu=True)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### memory_status

In [11]:
'{}/{}'.format(byte2str(psutil.virtual_memory().used),byte2str(psutil.virtual_memory().total))

'800.09M/38.99G'

### memory_percentage

In [36]:
psutil.virtual_memory().percent

10.4

In [39]:
def get_plist():
    p_list = []
    p_list_dict = {}
    for i in psutil.process_iter():
    #     print(i.as_dict())
        p_list.append((i.as_dict()['cpu_percent'],i.as_dict()['username'],i.as_dict()['name']))
        p_list_dict[str(i.as_dict()['pid'])]=(i.as_dict()['cpu_percent'],i.as_dict()['username'],i.as_dict()['name'])
    return p_list,p_list_dict
p_list,p_list_dict=get_plist()
print('{}:{}:{}%'.format(sorted(p_list)[-1][1],sorted(p_list)[-1][2],sorted(p_list)[-1][0]))
# print(sorted(p_list)[::-1])


agent_h:python:412.0%


In [22]:
from pynvml import *
nvmlInit()

In [28]:
from pynvml import *
nvmlInit()
print("Driver Version:", nvmlSystemGetDriverVersion())
deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvmlDeviceGetHandleByIndex(i)
    print("Device", i, ":", nvmlDeviceGetName(handle))
    print("Temp",i,";",nvmlDeviceGetTemperature(handle,0))
    print("gpu_util", i, ":", nvmlDeviceGetUtilizationRates(handle).gpu)
    print("gpu_mem_util", i, ":", 100*nvmlDeviceGetMemoryInfo(handle).used/nvmlDeviceGetMemoryInfo(handle).total)
    print("gpu_total_mem", i, ":", byte2str(nvmlDeviceGetMemoryInfo(handle).total))
    print("gpu_used_mem", i, ":", byte2str(nvmlDeviceGetMemoryInfo(handle).used))
    procs = nvmlDeviceGetComputeRunningProcesses(handle)
    
    mem = -1
    user = None
    for p in procs:
        if p.usedGpuMemory > mem:
            user = '{}:{}:{}'.format(p_list_dict[str(p.pid)][1],
                                     p_list_dict[str(p.pid)][2],
                                     byte2str(p.usedGpuMemory))
    print(user)


#     print(handle)


Driver Version: b'430.40'
Device 0 : b'GeForce RTX 2080 Ti'
Temp 0 ; 45
gpu_util 0 : 98
gpu_mem_util 0 : 61.68928767915785
gpu_total_mem 0 : 10.76G
gpu_used_mem 0 : 6.64G
agent_h:python:6.62G
Device 1 : b'GeForce RTX 2080 Ti'
Temp 1 ; 30
gpu_util 1 : 0
gpu_mem_util 1 : 0.15257130865345894
gpu_total_mem 1 : 10.76G
gpu_used_mem 1 : 16.81M
None


In [71]:
def get_gpu_status(p_list_dict):
    gpu_status_list = []
    nvmlInit()
    deviceCount = nvmlDeviceGetCount()
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        procs = nvmlDeviceGetComputeRunningProcesses(handle)
        mem = 0
        user = None
        for p in procs:
            if p.usedGpuMemory > mem:
                user = '{}:{}:{}'.format(p_list_dict[str(p.pid)][1],
                                         p_list_dict[str(p.pid)][2],
                                         byte2str(p.usedGpuMemory))
        status = {
                       'gpu_no': i,
                       'gpu_name': nvmlDeviceGetName(handle).decode('utf-8'),
                       'gpu_memory_status': '{}/{}'.format(byte2str(nvmlDeviceGetMemoryInfo(handle).used),byte2str(nvmlDeviceGetMemoryInfo(handle).total)),
                       'gpu_util': nvmlDeviceGetUtilizationRates(handle).gpu,
                       'gpu_memory': round(100*nvmlDeviceGetMemoryInfo(handle).used/nvmlDeviceGetMemoryInfo(handle).total,2),
                       'gpu_temperature': nvmlDeviceGetTemperature(handle,0),
                       'top_gpu_user': user,
                   }
        gpu_status_list.append(status)
    return gpu_status_list
print(get_gpu_status(p_list_dict))

[{'gpu_no': 0, 'gpu_name': 'GeForce RTX 2080 Ti', 'gpu_memory_status': '4.80G/10.76G', 'gpu_util': 100, 'gpu_memory': 44.61, 'gpu_temperature': 46, 'top_gpu_user': 'agent_h:python:4.78G'}, {'gpu_no': 1, 'gpu_name': 'GeForce RTX 2080 Ti', 'gpu_memory_status': '16.81M/10.76G', 'gpu_util': 0, 'gpu_memory': 0.15, 'gpu_temperature': 31, 'top_gpu_user': None}]


In [72]:
import time
def get_sys_info():
    sys_dic = {'name': 'MIB',
               'details': {'cpu':'XEON E5 6666','cpu_cores':8, 'memory':"16GB*4&2666MHz",'gpu':'RTX 2080 Ti*2'},
               'timestamp': 1932846198,
               'cpu_total': 0.73,
               'cpu_per_core': [0.0,0.43,0.45,0.56,0.89,0.01,0.01,0.02],
               'memory_status': '562M/38.0G',
               'memory_percentage': 0.07,
               'top_cpu_user': 'agent_h',
               'gpu_status':[
                   {
                       'gpu_no': 0,
                       'gpu_name': 'RTX 2080 Ti',
                       'gpu_memory_status': '12M/8119M',
                       'gpu_util': 0.78,
                       'gpu_memory': 0.23,
                       'gpu_temperature': 48,
                       'top_gpu_user': 'agent_n',
                   },
                   {
                       'gpu_no': 1,
                       'gpu_name': 'RTX 2080 Ti',
                       'gpu_memory_status': '3434M/8119M',
                       'gpu_util': 0.33,
                       'gpu_memory': 0.75,
                       'gpu_temperature': 78,
                       'top_gpu_user': 'alien_z',
                   }
               ]
              }
    sys_dic['timestamp']=time.time()
    sys_dic['cpu_total']=psutil.cpu_percent(interval=.5, percpu=False)
    sys_dic['cpu_per_core']=psutil.cpu_percent(interval=.5, percpu=True)
    sys_dic['memory_status']='{}/{}'.format(byte2str(psutil.virtual_memory().used),byte2str(psutil.virtual_memory().total))
    sys_dic['memory_percentage'] = psutil.virtual_memory().percent
    p_list,p_list_dict=get_plist()
    sys_dic['top_cpu_user']='{}:{}:{}%'.format(sorted(p_list)[-1][1],sorted(p_list)[-1][2],sorted(p_list)[-1][0])
    sys_dic['gpu_status']=get_gpu_status(p_list_dict)
    sys_dic['details']={'cpu':platform.processor(),
                        'cpu_cores':str(len(sys_dic['cpu_per_core'])),
                        'memory':byte2str(psutil.virtual_memory().total),
                        'gpu':[x['gpu_name'] for x in sys_dic['gpu_status']]
                                          
    }
    return sys_dic
    

In [73]:
get_sys_info()

{'name': 'MIB',
 'details': {'cpu': 'x86_64',
  'cpu_cores': '16',
  'memory': '38.99G',
  'gpu': ['GeForce RTX 2080 Ti', 'GeForce RTX 2080 Ti']},
 'timestamp': 1567782748.407056,
 'cpu_total': 29.7,
 'cpu_per_core': [12.2,
  12.0,
  100.0,
  2.0,
  100.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  13.7,
  12.0,
  12.0,
  14.0,
  13.7],
 'memory_status': '3.54G/38.99G',
 'memory_percentage': 10.6,
 'top_cpu_user': 'agent_h:python:435.7%',
 'gpu_status': [{'gpu_no': 0,
   'gpu_name': 'GeForce RTX 2080 Ti',
   'gpu_memory_status': '4.80G/10.76G',
   'gpu_util': 99,
   'gpu_memory': 44.61,
   'gpu_temperature': 46,
   'top_gpu_user': 'agent_h:python:4.78G'},
  {'gpu_no': 1,
   'gpu_name': 'GeForce RTX 2080 Ti',
   'gpu_memory_status': '16.81M/10.76G',
   'gpu_util': 0,
   'gpu_memory': 0.15,
   'gpu_temperature': 31,
   'top_gpu_user': None}]}

In [77]:
import platform
import json
import requests

In [80]:
dic = get_sys_info()
data = json.dumps(dic)
url = 'http://localhost:5000/add_machine'
header = {'Content-Type': "application/json"}
response = requests.post(url, data=data, headers=header)
print(response.content.decode('utf-8'))


{
  "message": "add machien info successfully", 
  "status": true
}



In [82]:
url = 'http://127.0.0.1:5000/get_machines'
response = requests.get(url)
print(response.content.decode('utf-8'))

[
  {
    "cpu_per_core": [
      76.5, 
      0.0, 
      0.0, 
      98.0, 
      0.0, 
      78.4, 
      2.0, 
      80.0, 
      2.0, 
      76.0, 
      78.0, 
      2.0, 
      78.0, 
      2.0, 
      77.6, 
      2.0
    ], 
    "cpu_total": 81.5, 
    "details": {
      "cpu": "x86_64", 
      "cpu_cores": "16", 
      "gpu": [
        "GeForce RTX 2080 Ti", 
        "GeForce RTX 2080 Ti"
      ], 
      "memory": "38.99G"
    }, 
    "gpu_status": [
      {
        "gpu_memory": 24.26, 
        "gpu_memory_status": "2.61G/10.76G", 
        "gpu_name": "GeForce RTX 2080 Ti", 
        "gpu_no": 0, 
        "gpu_temperature": 45, 
        "gpu_util": 100, 
        "top_gpu_user": "agent_h:python:2.59G"
      }, 
      {
        "gpu_memory": 0.15, 
        "gpu_memory_status": "16.81M/10.76G", 
        "gpu_name": "GeForce RTX 2080 Ti", 
        "gpu_no": 1, 
        "gpu_temperature": 32, 
        "gpu_util": 0, 
        "top_gpu_user": null
      }
    ], 
    "memory_percent

In [ ]:
data = json.dumps(dic)
url = 'http://127.0.0.1:5000/add_machine'
header = {'Content-Type': "application/json"}
response = requests.post(url, data=data, headers=header)
print(response.content.decode('utf-8'))
url = 'http://127.0.0.1:5000/get_machines'
response = requests.get(url)
print(response.content.decode('utf-8'))